In [40]:
import PySAM.Pvsamv1 as pv

In [41]:
pvmodel = pv.default('PVBatteryResidential')

In [42]:
# there are formatting differences between the weather file that we download from
# pvwatts and what SAM wants --- SAM wants the file from the NSRDB 
pvmodel.SolarResource.solar_resource_file = "../data/Test data for SAM.csv"

In [43]:
pvmodel.execute()